

```
Gerekli pip leri kurma
```



In [1]:
!pip install opencv-python
!pip install pandas
!pip install matplotlib
!pip install sentence-transformers

^C
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
zip_path = '/content/drive/MyDrive/obss-intern-competition-2025.zip'

!unzip -q /content/drive/MyDrive/obss-intern-competition-2025.zip -d /content/data/


In [5]:
train = pd.read_csv('/content/data/train.csv')
test = pd.read_csv('/content/data/test.csv')
submission = pd.read_csv('/content/data/sample_submission.csv')

In [6]:
print(train.head())
print(test.head())
print(submission.head())

   image_id                                            caption
0         0  The image features a comic-style panel depicti...
1         1  Colorful postcard featuring "Greetings from Ch...
2         2  Two vending machines display a variety of drin...
3         3  A man speaks at the eGovernment Conference 201...
4         4  A close-up of several silver coins stacked tog...
   image_id
0    100000
1    100001
2    100002
3    100003
4    100004
   image_id                                     caption
0    100000  your predicted caption for image_id 100000
1    100001  your predicted caption for image_id 100001
2    100002  your predicted caption for image_id 100002
3    100003  your predicted caption for image_id 100003
4    100004  your predicted caption for image_id 100004


In [7]:
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00


Modeli train görsel ve captionları ile eğitme

In [8]:
import torch
import open_clip
import pandas as pd
from PIL import Image
from torchvision import transforms
from tqdm import tqdm


train_df = pd.read_csv('/content/data/train.csv')

# open_clip modeli yükleniyor
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Sonuçları saklayacağımız liste
similarity_scores = []

# İlk 100 veri üzerinde dön
for i in tqdm(range(100)):
    try:
        image_id = train_df.loc[i, 'image_id']
        caption = train_df.loc[i, 'caption']
        image_path = f'/content/data/train/train/{image_id}.jpg'

        # Görseli oku ve işle
        image = Image.open(image_path).convert("RGB")
        image_input = preprocess(image).unsqueeze(0).to(device)

        # Caption'ı hazırla
        text_input = tokenizer([caption]).to(device)

        # Embedding’leri al
        with torch.no_grad():
            image_features = model.encode_image(image_input)
            text_features = model.encode_text(text_input)

        # Normalize et
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        # Cosine benzerliği hesapla
        similarity = (image_features @ text_features.T).item()
        similarity_scores.append((image_id, round(similarity, 4)))

    except Exception as e:
        print(f"Hata oluştu ({i}. örnek): {e}")

# Sonuçları kaydet
similarity_df = pd.DataFrame(similarity_scores, columns=["image_id", "similarity"])
similarity_df.to_csv("clip_similarity_scores.csv", index=False)
print("CLIP benzerlik puanları başarıyla kaydedildi.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

100%|██████████| 100/100 [00:43<00:00,  2.29it/s]

CLIP benzerlik puanları başarıyla kaydedildi.


In [9]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

Modelin test dosyası içindeki görsellere uyumlu caption'lar oluşturması ve bunları csv'ye kaydetmesi

In [ ]:
import torch
import pandas as pd
from PIL import Image
from tqdm import tqdm
import open_clip

# Model ve preprocess yükle
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Eğitim caption'larını oku
train_df = pd.read_csv('/content/data/train.csv')
captions_pool = train_df['caption'].tolist()

# Caption embedding'lerini hesapla
with torch.no_grad():
    text_inputs = tokenizer(captions_pool).to(device)
    text_features_all = model.encode_text(text_inputs)
    text_features_all /= text_features_all.norm(dim=-1, keepdim=True)

# Test verisini oku
test_df = pd.read_csv('/content/data/test.csv')
submission = []

for image_id in tqdm(test_df['image_id'].unique()):
    image_path = f'/content/data/test/test/{image_id}.jpg'
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Image açılırken hata (image_id: {image_id}): {e}")
        continue

    image_input = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarities = (image_features @ text_features_all.T).squeeze()
        best_idx = similarities.argmax().item()
        best_caption = captions_pool[best_idx]

    submission.append({'image_id': image_id, 'caption': best_caption})

    # Belleği temizle
    del image_input, image_features, similarities
    torch.cuda.empty_cache()

# Sonuçları kaydet
submission_df = pd.DataFrame(submission)
submission_df.to_csv('submission.csv', index=False)
